In [125]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()
from functools import reduce

/home/droman/Documents/stuff/dl_course/venv/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
train = pd.read_csv("./train_converted.csv")


In [46]:
train = train[train.is_duplicate==1]
short_train = train
short_train = short_train[['name_1','name_2']]

In [99]:
len(pd.read_csv("./train_converted.csv"))

497819

In [94]:
def get_same_companies_graph(companies_dataframe):
    result = [] # [[[a,b,c], {a: [b, c], b: [c]}, 3], [[d,e], {d: [e]}], 2]
    for i, pair in tqdm(companies_dataframe.iterrows()):
        first = pair['name_1']
        second = pair['name_2']
        if first != second:
            flag = True

            for list_same_companies in result:
                if first in list_same_companies[0]:
                    if second not in list_same_companies[0]:
                        list_same_companies[0].append(second)
                        try:
                            list_same_companies[1][first].append(second)
                        except:
                            list_same_companies[1].update({first: [second]})
                        list_same_companies[2]+=1
                        flag = False
                else:
                    if second in list_same_companies[0]:
                        if first not in list_same_companies[0]:
                            list_same_companies[0].append(first)
                            try:
                                list_same_companies[1][second].append(first)
                            except:
                                list_same_companies[1].update({second: [first]})
                            list_same_companies[2]+=1
                            flag = False
            if flag:
                result.append([[first, second], {first: [second]}, 1])
    return result



In [101]:
companies_graph = get_same_companies_graph(short_train)

3658it [00:00, 4388.58it/s]


In [85]:
len(list(filter(lambda x: len(x[0])>35, companies_graph)))

1

In [104]:
# изначальное количество дубликатов компаний
list(map(lambda x: x[2], companies_graph))[:10]

[7, 9, 9, 5, 10, 29, 1, 3, 1, 29]

In [87]:
most_longest_same_companies_list = (list(filter(lambda x: len(x[0])>35,companies_graph)))

In [88]:
most_longest_same_companies_list

[[['zeon  private limited',
   'zeonporation',
   'zeon trading shanghaico',
   'zeon medical inc',
   'zeon chemicals lp',
   'zeon',
   'zeon specialty materials',
   'zeon cscporation',
   'zeon medical',
   'zeon chemicals yonezawaltd',
   'zeon advance polymix',
   'zs elastomers',
   'zeon asia  sdn bhd',
   'zeon north',
   'zeon trading',
   'zeon nano technology ltd',
   'zeon kaseiltd',
   'zeon korea ltd',
   'zeon belts private',
   'zeon chemicals ltd',
   'zeon produtos quimicos do brasil',
   'zeon research vietnam ltd',
   'zeon asia pte ltd',
   'zeon advanced polymix',
   'zeon chemicals l p',
   'zeon kasei  sa de cv',
   'zeon chemicals europe limited',
   'zeon polymixrporation',
   'zeon do brasil',
   'zeon shinhwa',
   'zeon kasei',
   'zeon europe',
   'zeon rim ltd',
   'zeon fb ltd',
   'zeon manufacturing vietnam ltd',
   'zeon '],
  {'zeon  private limited': ['zeonporation'],
   'zeonporation': ['zeon trading shanghaico',
    'zeon medical inc',
    'zeon c

In [105]:
LIST_UNIQUE_COMPANIES_ID = 0
DICT_OF_RELATIONS_ID = 1
COUNT_OF_RELATIONS_ID = 2
EXTRA_RELATIONS_ID = 3
def get_extra_relations(inner_companies_graph):
    extra_relations = []
    for unique_companies in tqdm(inner_companies_graph):
        unique_companies.append([])
        companies = unique_companies[LIST_UNIQUE_COMPANIES_ID]
        company_relations = unique_companies[DICT_OF_RELATIONS_ID]
        for company in companies:
            try:
                relations = company_relations[company]
                excluded_companies = relations
            except:
                excluded_companies = []
            excluded_companies.append(company)
            for extra_relation_company in companies:
                if extra_relation_company not in excluded_companies:
                    extra_relations.append([company, extra_relation_company])
                    unique_companies[COUNT_OF_RELATIONS_ID]+=1
                    unique_companies[EXTRA_RELATIONS_ID].append([company, extra_relation_company])
    return extra_relations


In [ ]:
extra_relations = get_extra_relations(companies_graph)

In [110]:
# итоговое количество дубликатов компаний
list(map(lambda x: x[COUNT_OF_RELATIONS_ID], companies_graph))[:10]

[56, 90, 90, 30, 110, 870, 2, 12, 2, 870]

In [108]:
len(extra_relations)

143806

In [109]:
extra_relations[:30]

[['jx nippon oil  gas exploration', 'nippon oil exploration usa limited'],
 ['jx nippon oil  gas exploration', 'jx nippon oil  gas explorationporation'],
 ['jx nippon oil  gas exploration', 'nippon oil exploration limited'],
 ['jx nippon oil  gas exploration technical servicesporation',
  'jx nippon oil  gas exploration'],
 ['jx nippon oil  gas exploration technical servicesporation',
  'jx nippon oil  gas exploration ltd'],
 ['jx nippon oil  gas exploration technical servicesporation',
  'jx nippon oil  gas exploration limited'],
 ['jx nippon oil  gas exploration technical servicesporation',
  'jx nippon oil  gas explorationporation'],
 ['jx nippon oil  gas exploration technical servicesporation',
  'jx nippon oil  gas exploration  pty'],
 ['jx nippon oil  gas exploration ltd', 'jx nippon oil  gas exploration'],
 ['jx nippon oil  gas exploration ltd',
  'jx nippon oil  gas exploration technical servicesporation'],
 ['jx nippon oil  gas exploration ltd',
  'jx nippon oil  gas explorati

In [113]:
companies_graph.sort(key=lambda x: x[COUNT_OF_RELATIONS_ID])

In [132]:
list(map(lambda x: x[COUNT_OF_RELATIONS_ID], companies_graph))[-10:]

[919, 919, 920, 920, 920, 920, 920, 920, 922, 1260]

In [126]:
total_companies_number = reduce(lambda x1, x2: x1+x2, map(lambda x: x[COUNT_OF_RELATIONS_ID], companies_graph))
total_companies_number

152030

In [127]:
PERCENT_OF_COMPANIES_ID = 4
for company in companies_graph:
    company.append(100*(company[COUNT_OF_RELATIONS_ID]/total_companies_number))

In [131]:
list(map(lambda x: x[PERCENT_OF_COMPANIES_ID], companies_graph))[-10:]

[0.6044859567190687,
 0.6044859567190687,
 0.6051437216338881,
 0.6051437216338881,
 0.6051437216338881,
 0.6051437216338881,
 0.6051437216338881,
 0.6051437216338881,
 0.6064592514635269,
 0.8287837926724988]

In [145]:
def get_split_index(inner_companies_graph, input_percent=20):
    percent_count = 0
    for i in range(len(inner_companies_graph)-1, -1, -1):
        if percent_count >= input_percent:
            return i
        percent_count+=inner_companies_graph[i][PERCENT_OF_COMPANIES_ID]

In [146]:
split_index = get_split_index(companies_graph, 20)
split_index

1079

In [ ]:
short_train = pd.DataFrame(short_train, columns=['name_1', 'name_2'])

In [ ]:
short_train.to_csv('./data/abbreviation_data.csv', index=False)